In [ ]:
import pandas as pd
from dotenv import load_dotenv
import os
from yt_comment_scraper_util import YoutubeCommentScraper, CommentScraperUtil
from datetime import datetime
load_dotenv()
import csv

In [ ]:
api_key = os.getenv('YOUTUBE_API_1')  # make sure you have your API key in the .env file
scraper = YoutubeCommentScraper(api_key)

In [3]:
start_date = '2024-01-01'
channel = 'Fox_News'
scrape_iteration = 9

script_dir = os.getcwd()

path_to_link_csv = os.path.join(script_dir,f'../data/Raw Data/{channel}_links.csv')
comment_csv_load_path = os.path.join(script_dir,f'../data/Raw Data/{channel}_comments_{str(scrape_iteration)}.csv')
comment_csv_save_path = os.path.join(script_dir,f'../data/Raw Data/{channel}_comments_{str(scrape_iteration+1)}.csv')
path_to_error_logs = os.path.join(script_dir,f'../data/Error Logs/{channel}_{str(datetime.now())}.csv')


most_recent_video_id, previous_comments_df = CommentScraperUtil.get_earliest_video_id(comment_csv_load_path)
video_ids_to_scrape = CommentScraperUtil.load_video_ids(path_to_link_csv, start_date, most_recent_video_id)

In [ ]:
all_comments = []
errors = []
for count, video_id in enumerate(video_ids_to_scrape):
    print(f'Scraping video #{count} | Video Id: {video_id}')
    test = scraper.get_video_comments(video_id)
    comments_data, error =test
    if comments_data:
        all_comments.extend(comments_data)
    if error:
        print(f'Error for {video_id}')
        errors.append({'video_id': video_id, "error": error})
        if error ==  "Quota Exceeded":
            break
new_comments = pd.DataFrame(all_comments)
errors_df = pd.DataFrame(errors)

In [6]:

comments_df = new_comments
comments_df.to_csv(comment_csv_save_path, quoting=csv.QUOTE_NONNUMERIC, escapechar='\\', index=False, encoding='utf-8')
errors_df.to_csv(path_to_error_logs, index=False)

print(f"There are {comments_df.size} comments")